In [1]:
from pynput import mouse

from pynput.keyboard import Key, Controller as KeyboardController
from pynput.mouse import Button, Controller as MouseController
# import pynput

from time import sleep
import random as rand

# pynput demo controlls

### controlling the mouse

In [2]:
main_height = 1439
main_length = 2559

main_center = (main_length/2, main_height/2)


mouse = Controller()

# Read pointer position
print('The current pointer position is {0}'.format(
    mouse.position))

# Set pointer position
mouse.position = main_center
print('Now we have moved it to {0}'.format(
    mouse.position))

# Move pointer relative to current position
# mouse.move(5, -5)

# Press and release
mouse.press(Button.right)
mouse.release(Button.right)

# Double click; this is different from pressing and releasing
# twice on macOS
# mouse.click(Button.left, 2)

# Scroll two steps down
# mouse.scroll(0, 2)

# monitoring the mouse

from pynput import mouse

def on_move(x, y):
    print('Pointer moved to {0}'.format(
        (x, y)))

def on_click(x, y, button, pressed):
    print('{0} at {1}'.format(
        'Pressed' if pressed else 'Released',
        (x, y)))
    if not pressed:
        # Stop listener
        return False

def on_scroll(x, y, dx, dy):
    print('Scrolled {0} at {1}'.format(
        'down' if dy < 0 else 'up',
        (x, y)))

# Collect events until released
# with mouse.Listener(
#         on_move=on_move,
#         on_click=on_click,
#         on_scroll=on_scroll) as listener:
#     listener.join()

# ...or, in a non-blocking fashion:
# listener = mouse.Listener(
#     on_move=on_move,
#     on_click=on_click,
#     on_scroll=on_scroll)
# listener.start()

listener = mouse.Listener(
    on_move=on_move,
    on_click=on_click,
    on_scroll=on_scroll)
listener.start()

--------

# mouse listener

In [3]:
from pynput.mouse import Button, Controller

from pynput import mouse


def on_click(x, y, button, pressed):
    print(f'{pressed} at {x,y}')

listener = mouse.Listener(
    on_click=on_click,
)



In [4]:
listener.start()

In [5]:
listener.stop()

------------

# Keyboard Managment

In [6]:
keyboard = KeyboardController()

In [7]:
def basic_key_press(key):
    '''
    :param: key - string
                the non-special key to be pressed and released
    '''
    
    keyboard = KeyboardController()
    keyboard.press(f'{key}')
    # average human keypress time
    sleep(rand.uniform(0.08,0.15))
    keyboard.release(f'{key}')
    

In [8]:
def press_space():
    keyboard = KeyboardController()
    keyboard.press(Key.space)
    sleep(rand.uniform(0.07,0.1))
    keyboard.release(Key.space)
    


-----------------------

# Mouse Management

In [9]:
mouse = MouseController()

In [10]:
def move_mouse(pos):
    '''
    :param: pos - integer, tuple
            refers to the x,y pixel locations needed as 
            destination for mouse. Top Left corner of screen is (0,0)
    '''
    mouse = MouseController()

    mouse.position = pos
    print(f'Cursor moved to {mouse.position}. ')

In [11]:
def left_click():
    mouse = MouseController()
    mouse.click(Button.left)

In [12]:
def left_click_idle(n_clicks,sleep_time):
    '''
    clicks the current mouse position once about every :sleep_time: seconds
    for :n_clicks: amount of times
    
    :param: n_clicks - integer
            the total amount of clicks you want to execute before 
            the function ends
    
    :param: sleep_time - integer
            the rough amount in seconds that you want the 
            function to wait between clicks
            
            value is changed for randomness and ==
            random float between initial time & time*1.25
        
    '''
    mouse = MouseController()
    clicks = list(range(0,n_clicks+1))
    for i in clicks[1:]:
        y = rand.uniform(sleep_time, sleep_time * 1.25)
        sleep(y)
        mouse.click(Button.left)


------------

# Static component locations

### compass

In [13]:
compass = {}
compass['left'] = 2250
compass['right'] = 2270
compass['top'] = 48
compass['bottom'] = 68

#### 23rds screen inventory ####
##### slot 1
slot_1_left = 1346
slot_1_right = 1380
slot_1_bottom = 646
slot_1_top = 618

##### slot 2
slot_2_left = 1395
slot_2_right = 1430
slot_2_bottom = 646
slot_2_top = 620

##### slot 5
slot_5_left = 1346
slot_5_right = 1381
slot_5_bottom = 682
slot_5_top = 655

##### width gap
width_gap = 1395 - 1381

height_gap = 654 - 646

# In-Game Menu Construction

In [14]:
def make_inventory(screen_size):

    '''
    :param: screen_size
        string dtype - either "full" or "23rds"
        
        indicates the size of the inventory to build for other functions
    
    the inventory slots are squares and have an area.
    we want to make a bot that doesnt click the same pixel location for inventory slots so it's harder to track
    
    this function starts with the measurements for the inventory slots and uses the width/length & distance between cells
    to map out the x/y bounds for each of the 28 boxes
            
    --------------
    INVENTORY LAYOUT
    
    01 02 03 04
    05 06 07 08
    09 10 11 12
    13 14 15 16
    17 18 19 20
    21 22 23 24
    25 26 27 28
    --------------

    '''


    # estabilish main dictionary to house inventory locations
    inventory = {}

    # set starting pixel locations - Top left corner of monitor is 0,0
    # these are the starting locations of my particular inventory
    if screen_size == 'full':
        
        top = 612
        bottom = 641
        starting_left = 2201
        starting_right = 2235

        height_of_cell = bottom - top
        height_gap = 7

        width_of_cell = starting_right - starting_left
        width_gap = 15
        
    else:        
        top = 617
        bottom = 645
        starting_left = 1346
        starting_right = 1380

        height_of_cell = bottom - top
        height_gap = 6

        width_of_cell = starting_right - starting_left
        width_gap = 15

    # inventory is a grid, 7 down, 4 across - 28 total squares
    # cols makes a list of possible column numbers
    cols = list(range(1,28,4))


    for col in cols:
        
        # this resets the x values because each column starts withe same x-min/max
        if screen_size == 'full':
            left = starting_left
            right = starting_right
        else:
            left = starting_left
            right = starting_right
            
        
        # rows makes a list of the next 3 inv. slots based on the starting column
        rows = list(range(col, col+4,1))

        # for each row, set the values for the inventory 
        # if the row is between the starting col and the next 3 slots, do not change the y-value
        # else change the y-value and repeat

        for row in rows:
            # make new slot coordinates per row
            inventory[f'slot_{row}'] = {}

            if row < (col+3):
                inventory[f'slot_{row}']['left'] = left
                inventory[f'slot_{row}']['right'] = right
                inventory[f'slot_{row}']['top'] = top
                inventory[f'slot_{row}']['bottom'] = bottom

                # change the values of cells based on previous/current values
                # new values = prev values + change
                left = left + width_of_cell + width_gap
                right = left + width_of_cell

            else:

                inventory[f'slot_{row}']['left'] = left
                inventory[f'slot_{row}']['right'] = right
                inventory[f'slot_{row}']['top'] = top
                inventory[f'slot_{row}']['bottom'] = bottom

                # change the values of cells based on previous/current values
                # new values = prev values + change
                top = top + height_of_cell + height_gap
                bottom = top + height_of_cell
                left = left + width_of_cell + width_gap
                right = left + width_of_cell
    return inventory

In [15]:
def random_x_y(xmin, xmax, ymin, ymax):
    '''
    takes min and max, uses those values for generating a 
    random.uniform() integer.
    
    returns said integer as a tuple
    
    :return: example -> (int, int)
    
    '''
    
    x_random = rand.uniform(xmin, xmax)
    y_random = rand.uniform(ymin, ymax)
    
    return x_random, y_random
    
    
    

In [16]:
def menu_random_x_y(screen_size, player_menu, slot_num):
    '''
    menu must be constructed or passed in function param
    
    takes menu slot number (1 thru 36) as string
    
    returns random X,y in form of tuple as integers based 
    on either full screen or 2/3rds screen size
    to serve as position in mouse commands 
    
    :param: screen_size - string dtype
                    enter 'full' or '23rds'
    
    :param: slot_num - integer dtype
                    number that corresponds with the menu slot number
                    
    :param: player_menu - string dtype
                    menu that will be used as base for generating values
                    
    :return: (random_x, random_y)
    
    '''
    menu_dict = {}
    
    menu_dict['inventory'] = make_inventory(screen_size)
    menu_dict['spellbook'] = make_spellbook(screen_size)
    
    player_menu = menu_dict[f'{player_menu}']   
    x_random = rand.randrange(player_menu[f'slot_{slot_num}']['left'],
                                player_menu[f'slot_{slot_num}']['right'])

    y_random = rand.randrange(player_menu[f'slot_{slot_num}']['top'],
                                player_menu[f'slot_{slot_num}']['bottom'])

    random_x_y = (x_random, y_random)

    return random_x_y

In [17]:
def inventory_nav(screen_size, slot_start, slot_end, sleep_time, shuffle):
    
    '''
    take the start and end inventory locations, turn them into a list, then shuffle the list
    
    nav to random spot in random inventory slot and wait specified time
    
    
    :param: screen_size - string - either "full" or "23rds"
    :param: slot_start - inventory slot start location
    :param: slot_end - inventory slot end location
    :param: sleep_time - the amount of time in between clicks
    :param: shuffle - if 'True', shuffle inventory else, don't.
    
    --------------
    INVENTORY LAYOUT
    
    01 02 03 04
    05 06 07 08
    09 10 11 12
    13 14 15 16
    17 18 19 20
    21 22 23 24
    25 26 27 28
    --------------

    '''
    inventory = make_inventory(screen_size)    
    # make list of inv. slots
    inventory_slots = list(range(slot_start,slot_end+1))
    
    # shuffle inventory
    if shuffle == 'True':
        rand.shuffle(inventory_slots)
    else:
    
        #for each slot, nav to slot then wait +- 0.25% of sleep time
        for slot in inventory_slots:
            print(f'slot_{slot} -------------------- ')
            move_mouse(
                menu_random_x_y(screen_size,'inventory',slot))

            print()

            sleep(rand.uniform(
                (sleep_time*0.75),
                (sleep_time*1.25)))
    #         mouse.click(Button.left)

------------

# Spellbook

#### Spell book 23rds locations

#### Spell book slot 1
slot_1_left = 1553
slot_1_right = 1577
slot_1_bottom = 672
slot_1_top = 649

#### Spell book slot 2
slot_2_left = 1592
slot_2_right = 1615
slot_2_bottom = 671
slot_2_top = 650

#### Spell book slot 5
slot_5_left = 1553
slot_5_right = 1576
slot_5_bottom = 703
slot_5_top = 682

width = 23
width_gap = 15

height = 21
height_gap = 10


In [18]:
def make_spellbook(screen_size):

    '''
    :param: size
        string dtype - "23rds"
        
        indicates the size of the spellbook to build for other functions
    
    the spellbook slots are squares and have an area.
    we want to make a bot that doesnt click the same
    pixel location for spellbook slots so it's harder to track
    
    this function starts with the measurements for the 
    spellbook slots and uses the width/length & distance between cells
    to map out the x/y bounds for each of the 34 potential boxes
    
    The function will map out 36 squares as it maps in units of 4,
    so entering spell book slot numbers higher than the 
    corresponding sheet allows will cause the mouse to navigate
    to empty space

                        SPELLBOOK LAYOUT
    --------------   --------------  --------------  --------------
    Abilities        Combat           Teleport       Skilling
    
    01 02 03 04   |  01 02 03 04   |  01 02 03 04   | 01 02 03 04
    05 06 07 08   |  05 06 07 08   |  05 06 07 08   | 05 06 07 08
    09 10 11 12   |  09 10 11 12   |  09 10 11 12   | 09 10 11 12
    13 14 15 16   |  13 14 15 16   |  13 14 15 16   | 13 14 15 16
    17 18 19 20   |  17 18 19 20   |  17 18 19 20   | 17 18 19 20
    21 22 23 xx   |  21 22 23 xx   |  21 22 23 24   | xx xx xx xx
    xx xx xx xx   |  xx xx xx xx   |  25 26 27 28   | xx xx xx xx
    xx xx xx xx   |  xx xx xx xx   |  29 30 31 32   | xx xx xx xx
    xx xx         |  xx xx xx xx   |  33 34 xx xx   | xx xx xx xx
    --------------   --------------  --------------  --------------

    '''
 
    # estabilish main dictionary to house spellbook locations
    spellbook = {}

    # set starting pixel locations - Top left corner of monitor is 0,0
    # these are the starting locations of my particular spellbook
    if screen_size == 'full':
        
        top = 612
        bottom = 642
        starting_left = 2201
        starting_right = 2235

        height_of_cell = bottom - top
        height_gap = 7

        width_of_cell = starting_right - starting_left
        width_gap = 15
        
    else:        
        top = 646
        bottom = 671
        starting_left = 1553
        starting_right = 1577

        height_of_cell = bottom - top
        height_gap = 9

        width_of_cell = starting_right - starting_left
        width_gap = 15

    # spellbook is a grid, 9 down, 4 across - 36 total squares
    # cols makes a list of possible column numbers
    # cols = list of num, every 4 in 36
    cols = list(range(1,36,4))


    for col in cols:
        
        # this resets the x values because each column
        # starts withe same x-min/max
        if screen_size == 'full':
            left = starting_left
            right = starting_right
        else:
            left = starting_left
            right = starting_right
            
        
        # rows makes a list of the next 3 inv. slots based on the 
        # starting column
        rows = list(range(col, col+4,1))

        # for each row, set the values for the spellbook 
        # if the row is between the starting col and the next 3 slots,
        # do not change the y-value
        # else change the y-value and repeat

        for row in rows:
            # make new slot coordinates per row
            spellbook[f'slot_{row}'] = {}

            if row < (col+3):
                spellbook[f'slot_{row}']['left'] = left
                spellbook[f'slot_{row}']['right'] = right
                spellbook[f'slot_{row}']['top'] = top
                spellbook[f'slot_{row}']['bottom'] = bottom

                # change the values of cells based on previous/current
                # values
                # new values = prev values + change
                left = left + width_of_cell + width_gap
                right = left + width_of_cell

            else:

                spellbook[f'slot_{row}']['left'] = left
                spellbook[f'slot_{row}']['right'] = right
                spellbook[f'slot_{row}']['top'] = top
                spellbook[f'slot_{row}']['bottom'] = bottom

                # change the values of cells based on previous/current
                # values
                # new values = prev values + change
                top = top + height_of_cell + height_gap
                bottom = top + height_of_cell
                left = left + width_of_cell + width_gap
                right = left + width_of_cell
    return spellbook

In [19]:
def spellbook_nav(screen_size, slot_num):
    
    '''
    take the start and end spellbook locations, turn them into a list, then shuffle the list
    
    nav to random spot in random spellbook slot and wait specified time
    
    
    :param: screen_size - string - either "full" or "23rds"
    :param: slot_num - spellbook slot nav location
    
                        SPELLBOOK LAYOUT
    --------------   --------------  --------------  --------------
    Abilities        Combat           Teleport       Skilling
    
    01 02 03 04   |  01 02 03 04   |  01 02 03 04   | 01 02 03 04
    05 06 07 08   |  05 06 07 08   |  05 06 07 08   | 05 06 07 08
    09 10 11 12   |  09 10 11 12   |  09 10 11 12   | 09 10 11 12
    13 14 15 16   |  13 14 15 16   |  13 14 15 16   | 13 14 15 16
    17 18 19 20   |  17 18 19 20   |  17 18 19 20   | 17 18 19 20
    21 22 23 xx   |  21 22 23 xx   |  21 22 23 24   | xx xx xx xx
    xx xx xx xx   |  xx xx xx xx   |  25 26 27 28   | xx xx xx xx
    xx xx xx xx   |  xx xx xx xx   |  29 30 31 32   | xx xx xx xx
    xx xx         |  xx xx xx xx   |  33 34 xx xx   | xx xx xx xx
    --------------   --------------  --------------  --------------

    '''
    spellbook = make_spellbook(screen_size)    
    
    
    
    #for each slot, nav to slot then wait +- 0.25% of sleep time
    for slot in spellbook_slots:
        print(f'slot_{slot} -------------------- ')
        move_mouse(
            spellbook_random_x_y(
                spellbook_size,f'{slot}'))

        print()

        sleep(rand.uniform(
            (sleep_time*0.75),
            (sleep_time*1.25)))
#         mouse.click(Button.left)

---------------

# Actions

In [20]:
# smithing arrow tips 2/3rd screen

def item_smithing_23rds_screen():
    mouse = MouseController()

#     mouse.position = (rand.uniform(850,860), rand.uniform(1099,1104))
#     print(f'The current pointer position is {mouse.position}')
#     print()
#     mouse.click(Button.left)

    sleep(rand.uniform(1.25,1.53))

    mouse.position = (rand.uniform(921,925), rand.uniform(727,732))
    print(f'The current pointer position is {mouse.position}')
    print()
    mouse.click(Button.left)

    sleep(rand.uniform(1.2,1.57))

    press_space()
    
    print('============================================================')

In [21]:
def bar_smelting_23rds_screen(item_to_smelt, sleep_time, repeat):
    '''
    
    When standing directly in front of a smelter AFTER starting an inital
    smelt of one bar on 23rds screen, this function will perform idle 
    click commands for the player to make bars based on the time needed
    to smelted 60 units total OR the integer in place of the bar to smelt
    
    
    :param: item_to_smelt - string OR integer
    
            if string: the value will act as a key and pass an integer
                        crafting time to the function
            else integer: an exception will trigger and whatever integer
                        entered in for item_to_smelt will be passed to 
                        the function
                    
    :param: sleep_time - integer dtype
            
            The time between clicks of the mouse, will be converted to 
            random float using rand.uniform(sleep_time, sleep_time *1.25)
                    
                    
    :param: repeat - integer dtype
    
    '''
    
    mouse = MouseController()
    
    item_dict = {
        'gold':147,
        'necronium':108,
    }
    
    try:
        smelting_time = item_dict[f'{item_to_smelt}']
    except KeyError:
        smelting_time = item_to_smelt
    
    for i in range(0,repeat):


        smelter_x_min = 813
        smelter_x_max = 893
        smelter_y_min = 605
        smelter_y_max = 699

        mouse.position = (
            rand.uniform(
                smelter_x_min,
                smelter_x_max)
            ,
            rand.uniform(
                smelter_y_min,
                smelter_y_max))

        sleep(rand.uniform(sleep_time,sleep_time*1.25))

        left_click()
        
        sleep(rand.uniform(sleep_time,sleep_time*1.25))
        
        press_space()
        
        sleep(rand.uniform(smelting_time,smelting_time*1.05))
        
        print(f'finished cycle {i} of {repeat}')

In [22]:
def afk_mining(screen_size, repeat_cycles):
    '''
    
    :param: screen_size - string - 'full' or '23rds'
            used when generating / navigating the inventory

    
    :param: repeat_cycles - int - The number of 60 second cycles you
    want to repeat in which the function clicks 1st slot, then idle 
    clicks once per 6-ish seconds, 5 times before repeating
    
    
    using the position of the mouse at the point of activation,
    this function idly clicks every 5-6 seconds before clicking
    the first inventory slot to store ore in ore box and then returning
    initial posisiton.

    :param: initial_pos 
            
            The position of the mouse at the point of activation
    
    '''

    # set starting mouse pos
    initial_pos = mouse.position

    for i in range(0,repeat_cycles):
        print('--------------------------')

    # move to inventory slot
        move_mouse(
            menu_random_x_y(screen_size,'inventory',1))

    # wait
        sleep(rand.uniform(0.6,1.1))

    # click
        mouse.click(Button.left)

    # wait
        sleep(rand.uniform(0.25,0.5))

    # move to original position
        move_mouse(initial_pos)

    # wait
        sleep(rand.uniform(0.53,0.86))

    # click
        mouse.click(Button.left)
        
        print(f'inventory cycle {i} of {repeat_cycles}')
        print()
        
        # sit and click for about 30 seconds before repeating
        for j in range(0,5):
            
            # move to original position
            move_mouse(initial_pos)
            
            sleep (rand.uniform(1,1.25))
            
            left_click_idle(1,5)
            print(f'left_click_idle {j} of 5')
            print()
            



In [32]:
def lumbridge_jewelery_smithing(item_to_craft, repeat_cycles):
    
    '''
   
    :param: item_to_craft - string
            this item name will be used as a key in item dict to return
            an integer used as the wait time between crafting cycles
            
    :param: repeat_cycles - integer
            how many times you want to repeat the crafting cycle.
            
            
    :param:
    
    
    created using 23rds screen.
    
    Starting at the Smelter, begin crafting the desired jewelery
    then click 'cancel'.
    
    Once complete, have the skilling gems set to quick inventory F7.
    
    Player will move_mouse, then click bank chest.
    wait 6-8 seconds (travel)
    then press F7
    move_mouse, then click smelter
    wait 6-8 seconds (travel)
    then hit space and wait for the :item_to_craft: amount of seconds
    
    
    '''
#     item_dict['item_name'] = crafting time
    item_dict = {
        'dragonstone_necklace':52,
        'dragonstone_bracelet':52,
        
    }
    
    crafting_time = item_dict[f'{item_to_craft}']

    # when standing at the smelter these are the bank
    # chest X/Y

    bank_chest_x_min = 114
    bank_chest_x_max = 165
    bank_chest_y_min = 620
    bank_chest_y_max = 640

    # when at bank chest, these are the X/Y
    # for smelter

    smelter_x_min = 1645
    smelter_x_max = 1692
    smelter_y_min = 775
    smelter_y_max = 832

    #repeat loop starts here

    for i in range(0,repeat_cycles):
        # set chest coords
        random_chest_coordinate = random_x_y(
            bank_chest_x_min,bank_chest_x_max,
            bank_chest_y_min,bank_chest_y_max)

        # move mouse to chest
        move_mouse(random_chest_coordinate)

        # mouse.click(Button.left)
        left_click()
        
        # time to walk to chest
        sleep(rand.uniform(6,6*1.25))
        
        # set keyboard as controller input
        keyboard = KeyboardController()
        
        # press quick inv. command key
        keyboard.press(Key.f7)
        sleep(0.08)
        keyboard.release(Key.f7)
        
        # find smelter
        random_smelter_coordinate = random_x_y(
            smelter_x_min,smelter_x_max,
            smelter_y_min,smelter_y_max)
        sleep(0.15)
        # move mouse to smelter
        move_mouse(random_smelter_coordinate)
        sleep(0.12)
        # mouse.click(Button.left)
        left_click()
        
        # time to walk to smelter
        sleep(rand.uniform(6,6*1.25))
        
        # initialize keyboard and hit space
        press_space()
        
        # wait a random time between minimum crafting time and a 
        # lil more than that
        sleep(rand.uniform(crafting_time, crafting_time*1.065))

In [24]:
def blip_click(repeat_cycles):
    '''
    :param: repeat_cycles
    :param: position_to_click - tuple of int - (x,y) pixel coordinates
    
    
    using the position of the mouse at the point of activation,
    this function idly clicks every 5-6 seconds before clicking
    the first inventory slot to store ore in ore box and then returning
    initial posisiton.

    :param: starting_position 
            
            The position of the mouse at the point of activation
    
    '''
       
        
    # set starting mouse pos
    starting_position = mouse.position
    
    
    sleep(3)
    
    position_to_click = mouse.position

    for i in range(0,repeat_cycles):
        
        move_mouse(position_to_click)
    # click
        left_click_idle(1,0.5)
        
        move_mouse(starting_position)
        
        left_click_idle(1,0.5)
        
        sleep(10)


In [26]:
def item_enchanting(screen_size,enchant_spell,location,repeat_cycles):
    '''
    :param: screen_size - string
                either '23rds' or 'full', the value passed into
                navigation functions
                
    :param: enchant_spell - string
                the gemstone name for identifying the menu slot needed 
                to correctly navigate the mouse
    
    :param: location - string
                The location of player when standing.
                
    :param: repeat_cycles - int
                Number of times to repeat
                
    When standing in the proper location, fill inventory with first
    enchanting cycle then activate the bot with the quick inventory 
    selection set to option 1.
    
    '''
    # --------------------- spell slots & casting time ------------------ #

    # assign integer locations for Move_mouse function
    spell_slot_dict = {
        'sapphire':5,
        'emerald':8,
        'ruby':11,
        'diamond':14,
        'dragonstone':19,
        'onyx':20,
    }
    
    casting_time_dict = {
        'sapphire':34,
        'emerald':34,
        'ruby':34,
        'diamond':34,
        'dragonstone':34,
        'onyx':34,
    }
    
    
    # -------------------------- locations ------------------------------ #
    # set up dict to hold locations
    location_dict = {}
    
    # click the banker, then the exchange clerk for correct centering of
    # player
    location_dict['ge'] = {
        'left': 786,
        'right': 802,
        'bottom': 650,
        'top': 599,
    }
    
    # directly in front of lumbridge bank chest as if you were to open it
    location_dict['lumbridge'] = {
        'left' : 123,
        'right' : 123,
        'bottom' : 123,
        'top' : 123,
    }
    # ------------------------------------------------------------------- #

    # selects spell slot & castimg time based on string key
    # in function Kwarg
    spell_slot = spell_slot_dict[f'{enchant_spell}']
    casting_time = casting_time_dict[f'{enchant_spell}']

    # pick a random inventory slot, either 4 or 8
    inventory_slot = rand.choice([4,8])
    
    for i in range(0,repeat_cycles):
        print(f'Starting cycle {i+1} of {repeat_cycles}')
        print('---------------------------------------')
        
        # move mouse to bank chest/ banker, based on location KWARG
        move_mouse(random_x_y(
                    location_dict[f'{location}']['left'],
                    location_dict[f'{location}']['right'],
                    location_dict[f'{location}']['bottom'],
                    location_dict[f'{location}']['top'],))
        
        # sleep - human average reaction time
        sleep(rand.uniform(1.5,1.875))
        
        # open bank menu
        left_click()
        
        # wait for menu to load
        sleep(rand.uniform(1.8,2.23165))

        # press key for quick menu1
        basic_key_press('1')
        
        # sleep - human average reaction time
        sleep(rand.uniform(0.8,1.2645))
        
        # move mouse to a random location within the spellbook, spell area
        # based on Kwarg
        move_mouse(
            menu_random_x_y(screen_size,'spellbook',spell_slot)
        )

        # sleep - human average reaction time
        sleep(rand.uniform(0.56,0.858))

        # click the spell
        left_click()

        # sleep - human average reaction time
        sleep(rand.uniform(0.65,1.05))

        # move mouse to inventory 4 or 8
        # the closest slots to the spell and likely the most human choice
        move_mouse(menu_random_x_y(screen_size,'inventory',inventory_slot))

        # wait for menu to load
        sleep(rand.uniform(0.8,1.25))

        # click - inventory
        left_click()
        
        sleep(rand.uniform(1.2,1.455))
        
        # start the casting
        press_space()

        # sleep during casting time
        sleep(rand.uniform(casting_time, casting_time*1.056))
        print(f'Completed cycle {i+1} of {repeat_cycles}')
        print('---------------------------------------')
        



In [27]:
# high_alch(screen_size, repeat_cycles)

# need to set up inventory nav style repeat clicking?[
# should avoid point-to-point mouse movements
# develop after complex mouse mover.

In [ ]:
item_enchanting('23rds','dragonstone','ge',3)

Starting cycle 1 of 3
---------------------------------------
Cursor moved to (793, 636). 
Cursor moved to (1648, 795). 
Cursor moved to (1505, 668). 


In [ ]:
lumbridge_jewelery_smithing('dragonstone_bracelet',5)

Cursor moved to (134, 639). 
Cursor moved to (1686, 811). 


In [29]:
bar_smelting_23rds_screen('necronium', 1.25, 2)

In [ ]:
inventory_nav('23rds', 2,28,0.5, 'False')

In [ ]:
afk_mining('23rds',8)

In [ ]:
left_click_idle(20,6)

In [30]:
86807 / 110

789.1545454545454

# thoughts on creating a mouse line-mover

- need to have a means of calculating the 'line of a point to travel such that the mouse will move itself to a number of pixels between point A and B in hopes of fooling and trackers assigned to monitor player mouse movements.

- step one:
    calculate the starting point and ending point.
- step two:
    we need to calculate the line that exists between the two points

- step three: set a number of pexels to land the mouse dependind on the length of the line because the computer zips the mouse at lightspeed and humans cannot do that.
    - the number of travel points will have to be a large number of pixels
        perhaps as many as there are pixels along the diagonal line.
## Also: 
    - humans move in the swoops, not straight lines.
    maybe we can develop a polynomial line movement after impleentation of the pythagorean mouse movement function.
    
    
Summary:

plot starting point
plot ending point
plug into formula for a^2 + b^2 = c^2?
decide number of points to have mouse navigate

then test function.

# Bench Crafting

In [ ]:
# when standing infront of a crafting bench, w

menu_bar_x_min = 1268
menu_bar_x_max = 1461
menu_bar_y_min = 830
menu_bar_y_max = 850

crafting_bench_x_min = 1248
crafting_bench_x_max = 1294
crafting_bench_y_min = 538
crafting_bench_y_max = 573

mouse.position = (rand.uniform(1248,1294), rand.uniform(538,573))
print(f'The current pointer position is {mouse.position}')
print()
mouse.click(Button.left)


sleep(rand.uniform((sleep_time),(sleep_time*1.25)))


mouse.position = (rand.uniform(1268,1461), rand.uniform(830,850))
print(f'The current pointer position is {mouse.position}')
print()
mouse.click(Button.left)

In [ ]:

for i in range(0,3):
    
    mouse.position = (rand.uniform(1248,1294), rand.uniform(538,573))
    print(f'The current pointer position is {mouse.position}')
    print()
    mouse.click(Button.left)
    sleep(rand.uniform(52,53.25))
    mouse.position = (rand.uniform(1268,1461), rand.uniform(830,850))
    print(f'The current pointer position is {mouse.position}')
    print()
    mouse.click(Button.left)
    
    sleep(rand.uniform(52,53.25))
    mouse.position = (rand.uniform(1268,1461), rand.uniform(830,850))
    print(f'The current pointer position is {mouse.position}')
    print()
    mouse.click(Button.left)
    

----------

make a listener to activate on right click so you can record where the mouse needs to be.

define the area for all static components
- compass
- inventory slots (1-28)
- certain menu bars
- bank icon locations
- spell locations in magic menu

build a method to subtract a random number between a particular range of the x/y min/max so that it clicks a space within the set bounds but not the same place each time

build a click and drag method that takes start and stop tuples with no return

build a click method that records starting location, then clicks a specified location before returning to starting pointaa

- build a function that uses the start and stop x/y location for two points then makes a line with thos two points
    - humans dotn teleport the mouse where they need it to be
        - have a method that accepts the start and stop locations for where you want the action to take place
            then use the two known values to calculate y=mx+b and travel z points along that line
            z will == a number of points defined by the length of the line in pixels
            
            
- build a listener function using key board that will open/access menus with certain key presses/ combination keys
    utilize Blip_click to preseve mouse location between actions.
    listener: while shift+key is pressed, do action based on the resulting key - like teleporting or opening a menu

chekc the class style build from geeks for geeks

define x and y min, max  for inventory
then define the area for each cell and the distance between the pixels of each cell that leave it clickable/useable

then perform actions as needed.

The goal is to make the movements as non-robotic as possible.
incorparate rand and create a method that splits the difference between the two points then navigates to those points without clicking the same pixel in the same tiem window.

the bot needs to be able to navigate to compass, then L click, then press W for  0.75-1.25 sec
and use that as the screen 

# ideas:

making bone 2 banana bot

make empty inventory bot

drop inventory bot